In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader


C:\Users\91880\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# loaded all pdfs data


In [ ]:
def load_all_pdfs_separately(pdf_folder_path):
    """
    Load PDFs from a folder, keeping each PDF separate.
    Returns a dictionary: {pdf_filename: [list of Document pages]}
    """
    pdf_documents = {}  # dictionary to store pages per PDF

    for file_name in os.listdir(pdf_folder_path):
        if file_name.endswith(".pdf"):
            full_path = os.path.join(pdf_folder_path, file_name)
            print(f"Loading: {file_name}")

            loader = PyPDFLoader(full_path)
            documents = loader.load()

            # Add metadata for source PDF
            for doc in documents:
                doc.metadata["source"] = file_name

            # Store documents under their PDF filename
            pdf_documents[file_name] = documents

    return pdf_documents


# Path to your PDF folder
pdf_folder_path = r"C:\Users\91880\Desktop\GenAI_Projects\PuchoHR\Data\documents"

# Load PDFs separately
pdf_docs = load_all_pdfs_separately(pdf_folder_path)

# Example output
for pdf, pages in pdf_docs.items():
    print(f"{pdf}: {len(pages)} pages")


Loading: biology.pdf
Loading: Blockchain.pdf
Loading: Cybersecurity.pdf
Loading: quantum.pdf
biology.pdf: 107 pages
Blockchain.pdf: 33 pages
Cybersecurity.pdf: 1067 pages
quantum.pdf: 438 pages


In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [22]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap=256)
pdf_chunks_dict={}

for pdf_file, pages in pdf_docs.items():
    chunks_for_pdf=text_splitter.split_documents(pages)
    pdf_chunks_dict[pdf_file]=chunks_for_pdf

In [ ]:
#just checking chunk has been done properly 

first_pdf = list(pdf_chunks_dict.keys())[0]
print(f"Checking first 3 chunks of {first_pdf}:\n")

for i in range(3):
    print(f"Chunk {i}:")
    print(pdf_chunks_dict[first_pdf][i].page_content[:300], "...")  # show first 300 chars
    print("Metadata:", pdf_chunks_dict[first_pdf][i].metadata)
    print("-"*50)


Checking first 3 chunks of biology.pdf:

Chunk 0:
Synthetic Biology Textbook ...
Metadata: {'producer': 'Skia/PDF m97 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Synthetic Biology Textbook.docx', 'source': 'biology.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}
--------------------------------------------------
Chunk 1:
Thepassionfor thepurposeof CBSinissomethingthatwediscoveredrightawayinthefirstcontact withtheacademicentity. EveryonewhoenterstheClubhasthesoledesiretolearnSynthetic Biology, but when inside, we are surrounded bywelcomingpeoplethat makeuslove what we do. Every task becomes a passion and it is perfor ...
Metadata: {'producer': 'Skia/PDF m97 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Synthetic Biology Textbook.docx', 'source': 'biology.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}
--------------------------------------------------
Chunk 2:
task becomes a passion and it is performed by interest i

#Embeed the chunks

In [28]:
from langchain_ollama import OllamaEmbeddings


In [37]:
embedding=OllamaEmbeddings(
    model="nomic-embed-text"
)

#store in chroma db

In [32]:
from langchain_community.vectorstores import Chroma


In [ ]:
#Chroma expects a list of Documents, not a dictionary so changing it to document

all_chunks=[]
for chunks in pdf_chunks_dict.values():
    all_chunks.extend(chunks)
print("total chunks",len(all_chunks))

total chunks 21121


In [3]:
#create chroma vector db


vector_db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding
)

NameError: name 'Chroma' is not defined